# Luminosity Analysis

In this notebook I will complete an analysis of stellar luminosity within the SMC Wing.

In [1]:
# Standard imports

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pylab import *
import scipy as scipy
from scipy import stats
import scipy.integrate as integrate
import itertools
from astropy.io import fits

## Field star photometry without applying any color / (NUV)o thresholds to data

The following sample includes 1104 stars.

In [2]:
# Original files
SMC_daofind_braun_id_coords_color_nuv = np.genfromtxt('SMC_daofind_braun_id_coords_color_nuv')

# Uncorrected variables
id_number_nocorr = SMC_daofind_braun_id_coords_color_nuv[:,0]
ra_nocorr = SMC_daofind_braun_id_coords_color_nuv[:,1]
dec_nocorr = SMC_daofind_braun_id_coords_color_nuv[:,2]
nuv_v_nocorr = SMC_daofind_braun_id_coords_color_nuv[:,3]
nuv_nocorr = SMC_daofind_braun_id_coords_color_nuv[:,4]

# Corrected variables (including absolute mag correction and extinction correction)
id_number_corr = id_number_nocorr
ra_corr = ra_nocorr
dec_corr = dec_nocorr
nuv_v_corr = nuv_v_nocorr - 0.26
nuv_corr = nuv_nocorr - 18.7 - 0.42
v_corr = nuv_corr - nuv_v_corr # The v extinction correction of - 0.16 is "built in" to the defintion of the nuv_v color.

# Brightness cut
id_number_cut = id_number_corr[nuv_corr <= 0.45]
ra_cut = ra_corr[nuv_corr <= 0.45]
dec_cut = dec_corr[nuv_corr <= 0.45]
nuv_v_cut = nuv_v_corr[nuv_corr <= 0.45]
nuv_cut = nuv_corr[nuv_corr <= 0.45]
v_cut = v_corr[nuv_corr <= 0.45]

# Correction between GALEX catalog vs. daophot
id_number = id_number_cut
ra = ra_cut
dec = dec_cut
nuv_v = nuv_v_cut + 0.6
nuv = nuv_cut + 0.6
v = v_cut

## Individual stellar luminosity from observed NUV magnitude

Individual stellar luminosities were calculated as a direct conversion from the DAOPHOT photometry analysis in AB mags to counts / sec.

In [3]:
# Convert NUVo absolute magnitudes to counts per second

# Convert from extinction- and catalog-corrected absolute magnitudes to uncorrection AB magnitudes
    # 18.70 for conversion from absolute to apparent, 0.42 for extinction.
    # We leave in the 0.6 for catalog correction
nuv_abmag = nuv + 18.70 + 0.42

# Convert from AB magnitudes to counts per second (CPS) for 1104 stars
total_counts_stars = np.sum(10**((nuv_abmag-20.08)/-2.5))
print ("Total [cts/s] for 1104 stars:", np.round(total_counts_stars, 2))

Total [cts/s] for 1104 stars: 11958.34


## Region stellar luminosity from IRAF Apphot analysis

In order to determine the total observed stellar luminosity for our region, we completed a polygonal IRAF Apphot analysis surrounding the stellar sample, as shown in the image below.

<tr>
    <td> <img src="Jul15_1104_clusters_composite_stars.png" style="width: 400px;"/> </td>
    <td> Used composite GALEX image ngc602.fits and regions with <br >
        Jul15_tetris_clusters_composite.reg for photometry.<br ><br >
        <b>Region Parameters [ Tetris Region ]:</b> <br >
        <b>area_study</b> = 2139950 arcsec<sup>-2</sup><br >
        <b>sum_study</b> = 27342 cts/s<br >
        <b>mean_intensity_study [I_study]</b> = 0.01278 cts/s/arcsec<sup>2</sup><br ><br >
        <b>Background Estimate:</b> <br >
        Based on regions to the east and south of our zone.<br >
        <b>mean_intensity_bkgrnd [I_bkgrnd]</b> = 0.00305 cts/sec/arcsec<sup>2</sup><br >
        +/- 0.0005 cts/sec/arcsec<sup>2</sup><br ><br >
        <b>Background-Corrected Luminosity:</b> <br >
        <b>[I_corr = I_study - I_bkgrnd]</b> = 0.00973 cts/s/arcsec<sup>2</sup><br >
        <b>corrected_count_rate </b> = [I_corr] \* area_study = 20822 cts/s <br >
        +/- 15% statistical error --> real uncertainty probably ~25%<br ><br >
        <b>Comparison to Individual Stellar Luminosity:</b> <br >
        <b>Total [cts/s] for 1104 stars:</b> 11,958<br >
        <b>Corrected [cts/s] for Studied Region:</b> 20,822<br >
        <b>Unaccounted [cts/s] in Indiv. Stellar Lum.:</b> 8,864 (42%)
   </td>
</tr>

## Cluster stellar luminosity from IRAF Apphot analysis

In order to get an accurate reading about the luminosity in clusters, we need to consider both the stars that were included in the 1104-star sample calculated above (shown in blue above) and the cluster regions (shown in red above). Below, we outline the process of calculating the photometry of these two populations.

## Luminsoity in individual stars (shown in blue above)

In [4]:
composite_stars_coords = np.genfromtxt('Jul15_composite_stars_coords')
composite_stars_ra = composite_stars_coords[:,0]
composite_stars_dec = composite_stars_coords[:,1]

# Match stars by their RA values to the 1104-star sample
indices_in_clusters = np.nonzero(np.in1d(ra,composite_stars_ra))[0]

# Convert from AB magnitudes to counts per second (CPS) for stars in clusters (194)
nuv_in_clusters = nuv[indices_in_clusters]
nuv_abmag_in_clusters = nuv_in_clusters + 18.70 + 0.42
counts_clusters_indiv = np.sum(10**((nuv_abmag_in_clusters-20.08)/-2.5))
       
# Calculate luminosity in individual stars that were NOT included in the clusters (910)
indices_not_in_clusters = [x for x in range(len(nuv)) if x not in indices_in_clusters]
nuv_not_in_clusters = nuv[indices_not_in_clusters]

nuv_abmag_not_in_clusters = nuv_not_in_clusters + 18.70 + 0.42
total_counts_stars_not_in_clusters = np.sum(10**((nuv_abmag_not_in_clusters-20.08)/-2.5))
# Note: When working with individual stellar luminosity (as done here) we never need to correct with a
    # background subtraction. Because the photometry relies on assigned PSFs, we inevitably underestimate
    # the photometered luminosity and include minimal background light.

## Luminosity in cluster regions (shown in red above)
The file Jul15_cluster_analysis_7 contains the following columns: <br>
**RegNumber** **CenterRA** **CenterDec** **Sum**[cts/sec] **Error**[cts/sec] **Area**[arcsec<sup>2</sup>] **Surf_Bri**[sum/arcsec<sup>2</sup>] **Surf_Error**[sum/arcsec<sup>2</sup>]

In [5]:
cluster_analysis = np.genfromtxt('Jul15_cluster_analysis_7')

# The multiplication factor of 0.575 is analogous to the catalog correction in NUVo magnitudes.
counts_clusters = cluster_analysis[:,3] * 0.575 # cts/s
area_clusters = cluster_analysis[:,5] # arcsec**2

total_counts_clusters = np.sum(counts_clusters) # cts/s
total_area_clusters = np.sum(area_clusters) # arcsec**2

mean_intensity_clusters = counts_clusters/area_clusters # cts/s/arcsec**2

# I_corr = I_cluster - I_bkgrd 
# ...where I_bkgrd = 0.00305 cts/s/arcsec**2, as defined above.
mean_intensity_clusters_bkgrndcorr = mean_intensity_clusters - 0.00305

count_rate_clusters_bkgrndcorr = mean_intensity_clusters_bkgrndcorr * area_clusters
counts_clusters_region = np.sum(count_rate_clusters_bkgrndcorr)

total_counts_tetris = 20822

## Comparing cluster luminosity to that of the entire studied region

In [7]:
# Individual stellar luminosity
print ("Individual-to-individual analysis:")
print ("Total [cts/s] for individual stars in clusters:", np.round(counts_clusters_indiv, 2))
print ("Percentage of luminosity in clusters for individual stellar luminosity:", np.round(counts_clusters_indiv/total_counts_stars, 4) * 100)
print (" ")

# Region stellar luminosity
print ("Region-to-region analysis:")
print ("Total [cts/s] for cluster regions:", np.round(counts_clusters_region, 2))
print ("Percentage of luminosity in clusters for region stellar luminosity:", np.round(counts_clusters_region/total_counts_tetris, 4) * 100)


Individual-to-individual analysis:
Total [cts/s] for individual stars in clusters: 2454.68
Percentage of luminosity in clusters for individual stellar luminosity: 20.53
 
Region-to-region analysis:
Total [cts/s] for cluster regions: 5094.29
Percentage of luminosity in clusters for region stellar luminosity: 24.47


## Analysis

It is logical that we see a lower percentage of luminosity in clusters when considering individual stellar luminosity versus region stellar luminosity. Because clusters experience significant blending, the IRAF photometry could not resolve many of the individual stars within clusters. This results in a lower-bound percentage of luminosity in clusters based on the individual stellar luminosity analysis. The region analysis, which does not face deblending challenges, thus presents a more accurate analysis of the percentage of luminosity in clusters.